In [18]:
# import libraries
import pandas as pd
import yfinance as yf
import hvplot.pandas
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [19]:
# download the data
pltr_df = yf.download(tickers = 'PLTR', period = '5Y')
pltr_df

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,PLTR,PLTR,PLTR,PLTR,PLTR
Date,,,,,
2020-09-30,9.500000,11.410000,9.110000,10.000000,338584400
2020-10-01,9.460000,10.100000,9.230000,9.690000,124297600
2020-10-02,9.200000,9.280000,8.940000,9.060000,55018300
2020-10-05,9.030000,9.490000,8.920000,9.430000,36316900
2020-10-06,9.900000,10.180000,8.900000,9.040000,90864000
...,...,...,...,...,...
2025-05-30,131.779999,131.940002,122.750000,123.529999,185897600
2025-06-02,132.039993,134.479996,128.860001,131.434998,93218700


In [20]:
# drop the level with the ticker in the columns of the data frame
pltr_df = pltr_df.droplevel(level = 1, axis = 1)

# Remove the name 'Price' from the dataframe's columns
pltr_df.columns.name = None

# view data
pltr_df

,Close,High,Low,Open,Volume
Date,,,,,
2020-09-30,9.500000,11.410000,9.110000,10.000000,338584400
2020-10-01,9.460000,10.100000,9.230000,9.690000,124297600
2020-10-02,9.200000,9.280000,8.940000,9.060000,55018300
2020-10-05,9.030000,9.490000,8.920000,9.430000,36316900
2020-10-06,9.900000,10.180000,8.900000,9.040000,90864000
...,...,...,...,...,...
2025-05-30,131.779999,131.940002,122.750000,123.529999,185897600
2025-06-02,132.039993,134.479996,128.860001,131.434998,93218700
2025-06-03,133.169998,135.279999,130.100006,133.070007,91297700


In [21]:
# Copy pltr_df to a new dataframe for further analysis
signals_df = pltr_df[['Close']]

# visulise the data
signals_df.hvplot()

:Curve   [Date]   (Close)

In [22]:


# Set the long and short windows
short_window = 20
long_window = 50

# Obtain the Exponential Moving Average of the Close prices with short and long windows
signals_df['EMA20_Close'] =  round(signals_df['Close'].ewm(span = short_window).mean(), 2)
signals_df['EMA50_Close'] =  round(signals_df['Close'].ewm(span = long_window).mean(), 2)


# view data
signals_df.head()

,Close,EMA20_Close,EMA50_Close
Date,,,
2020-09-30,9.50,9.50,9.50
2020-10-01,9.46,9.48,9.48
2020-10-02,9.20,9.38,9.38
2020-10-05,9.03,9.28,9.29
2020-10-06,9.90,9.43,9.42


In [23]:
# Obtain the points of buy and sell using the 20 and 50 day exponential moving averages
buy_points = (signals_df['EMA20_Close'] > signals_df['EMA50_Close']) & (signals_df['EMA20_Close'].shift(1) <= signals_df['EMA50_Close'].shift(1))
sell_points = (signals_df['EMA20_Close'] < signals_df['EMA50_Close']) & (signals_df['EMA20_Close'].shift(1) >= signals_df['EMA50_Close'].shift(1))

# Combine the buy and sell points to obtain the all the crossover points of the EMAs
crossover_points = buy_points | sell_points

# Obtain the first crossover point
first_buy_point = crossover_points.idxmax()

# Mark the crossover points of the EMAs with 1s 
signals_df['Signal'] = np.where((signals_df['EMA20_Close'] > signals_df['EMA50_Close']) & 
                                         (signals_df.index > first_buy_point), 1, 0)

# View data
signals_df.head()

,Close,EMA20_Close,EMA50_Close,Signal
Date,,,,
2020-09-30,9.50,9.50,9.50,0
2020-10-01,9.46,9.48,9.48,0
2020-10-02,9.20,9.38,9.38,0
2020-10-05,9.03,9.28,9.29,0
2020-10-06,9.90,9.43,9.42,1


In [24]:
# Label the exit and entry points with Buy as 1, Sell as -1 and Hold as 0
signals_df['Entry/Exit'] = signals_df['Signal'].diff()
signals_df.dropna(inplace = True)

signals_df.head()

,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit
Date,,,,,
2020-10-01,9.46,9.48,9.48,0,0.0
2020-10-02,9.20,9.38,9.38,0,0.0
2020-10-05,9.03,9.28,9.29,0,0.0
2020-10-06,9.90,9.43,9.42,1,1.0
2020-10-07,10.00,9.55,9.53,1,0.0


In [55]:
# Create a function to obtain the dataframe with the dates around the trades alone
def subset_crossover(df, crossovers):
    crossindex = np.where(crossovers)[0]
    row_ranges = []
    for index in crossindex:
        start = max(index-1, 0)
        end = min(index+1, len(df))
        row_ranges.extend(range(start, end))
        
    unique_rows = sorted(set(row_ranges))
    
    return df.iloc[unique_rows]


In [57]:
# Call the function to create the dataframe with only the dates around the trades
crossovers_df = subset_crossover(signals_df, crossover_points)
crossovers_df

,Close,EMA20_Close,EMA50_Close,Signal,Entry/Exit
Date,,,,,
2020-10-05,9.030000,9.28,9.29,0,0.0
2020-10-06,9.900000,9.43,9.42,1,1.0
2020-10-07,10.000000,9.55,9.53,1,0.0
2020-10-20,9.270000,9.61,9.62,0,-1.0
2020-10-21,9.200000,9.57,9.58,0,0.0
2020-10-27,10.950000,9.76,9.71,1,1.0
2020-10-28,10.850000,9.88,9.78,1,0.0
2021-03-03,23.590000,27.58,27.67,0,-1.0
2021-03-04,24.730000,27.31,27.55,0,0.0


In [26]:
# Visualise the data with buy and sell points marked on the chart with the close prices
close_prices_df = signals_df['Close'].hvplot(color = 'lightgray')

ema20  = signals_df['EMA20_Close'].hvplot(color = 'green')

ema50  = signals_df['EMA50_Close'].hvplot(color = 'yellow')

entry = signals_df[signals_df['Entry/Exit'] == 1]['Close'].hvplot.scatter(color = 'blue',
                                                                 marker = '^',
                                                                 legend = False,
                                                                 size = 200
                                                                )

exit = signals_df[signals_df['Entry/Exit'] == -1]['Close'].hvplot.scatter(color = 'red',
                                                                 marker = 'v',
                                                                 legend = False,
                                                                 size = 200
                                                                )


plot = close_prices * ema20 * ema50 * entry * exit

plot.opts(height = 500,
         width = 1000,
         title = 'Entry Exit plot based on EMA',
         ylabel = 'Price in $')

:Overlay
   .Curve.Close       :Curve   [Date]   (Close)
   .Curve.EMA20_Close :Curve   [Date]   (EMA20_Close)
   .Curve.EMA50_Close :Curve   [Date]   (EMA50_Close)
   .Scatter.Close.I   :Scatter   [Date]   (Close)
   .Scatter.Close.II  :Scatter   [Date]   (Close)